# after initial EDA, lets train xgboost model with following set of features

In [35]:
# import necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import numpy as np
from sklearn.metrics import *
import xgboost as xgb
%matplotlib inline
sns.set()

In [2]:
# import datasets
st = datetime.datetime.now()
train_data = pd.read_csv("../data/train.csv")
test_data = pd.read_csv("../data/test_QkPvNLx.csv")
print("total time req to load data ",datetime.datetime.now() - st)

total time req to load data  0:00:00.820740


In [17]:
input_features = ["Course_Domain","Course_Type","Short_Promotion","Public_Holiday"]
target = ["Sales"]
categorical_features = ["Course_Domain","Course_Type"]
input_cols = ["Course_Domain","Course_Type","Short_Promotion","Public_Holiday","Sales"]

## One Hot Categorical Features

### combine training and testing data before one hot encoding

In [5]:
train_data["Flag"] = "train"
test_data["Flag"] = "test"
train_test_data = pd.concat([train_data,
                            test_data])
train_test_data.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  after removing the cwd from sys.path.


(548087, 12)

In [6]:
train_data.shape[0]+test_data.shape[0],train_test_data.shape

(548087, (548087, 12))

In [10]:
train_test_data.columns

Index(['Competition_Metric', 'Course_Domain', 'Course_ID', 'Course_Type',
       'Day_No', 'Flag', 'ID', 'Long_Promotion', 'Public_Holiday', 'Sales',
       'Short_Promotion', 'User_Traffic'],
      dtype='object')

In [13]:
train_data.isnull().sum()

ID                       0
Day_No                   0
Course_ID                0
Course_Domain            0
Course_Type              0
Short_Promotion          0
Public_Holiday           0
Long_Promotion           0
User_Traffic             0
Competition_Metric    1764
Sales                    0
Flag                     0
dtype: int64

In [14]:
test_data.isnull().sum()

ID                      0
Day_No                  0
Course_ID               0
Course_Domain           0
Course_Type             0
Short_Promotion         0
Public_Holiday          0
Long_Promotion          0
Competition_Metric    120
Flag                    0
dtype: int64

In [12]:
train_test_data.isnull().sum()

Competition_Metric     1884
Course_Domain             0
Course_ID                 0
Course_Type               0
Day_No                    0
Flag                      0
ID                        0
Long_Promotion            0
Public_Holiday            0
Sales                 36000
Short_Promotion           0
User_Traffic          36000
dtype: int64

In [18]:
st = datetime.datetime.now()
one_hot_train_test_data = pd.get_dummies(train_test_data,columns=categorical_features)
print("Time taken for One hot encoding ",datetime.datetime.now() - st)
one_hot_train_test_data.shape


Time taken for One hot encoding  0:00:00.175495


(548087, 17)

In [19]:
one_hot_train_test_data

,Competition_Metric,Course_ID,Day_No,Flag,ID,Long_Promotion,Public_Holiday,Sales,Short_Promotion,User_Traffic,Course_Domain_Business,Course_Domain_Development,Course_Domain_Finance & Accounting,Course_Domain_Software Marketing,Course_Type_Course,Course_Type_Degree,Course_Type_Program
0,0.007,1,1,train,1,1,1,81.0,0,11004.0,0,1,0,0,1,0,0
1,0.007,1,2,train,2,1,0,79.0,0,13650.0,0,1,0,0,1,0,0
2,0.007,1,3,train,3,1,0,75.0,0,11655.0,0,1,0,0,1,0,0
3,0.007,1,4,train,4,1,0,80.0,0,12054.0,0,1,0,0,1,0,0
4,0.007,1,5,train,5,1,0,41.0,0,6804.0,0,1,0,0,1,0,0
5,0.007,1,6,train,6,1,0,62.0,0,10395.0,0,1,0,0,1,0,0
6,0.007,1,7,train,7,1,0,122.0,1,16023.0,0,1,0,0,1,0,0
7,0.007,1,8,train,8,1,0,114.0,1,14385.0,0,1,0,0,1,0,0
8,0.007,1,9,train,9,1,0,121.0,1,16485.0,0,1,0,0,1,0,0
9,0.007,1,10,train,10,1,0,100.0,1,13377.0,0,1,0,0,1,0,0


In [20]:
# separate one hot train test data

one_hot_train_data = one_hot_train_test_data[one_hot_train_test_data.Flag=="train"]
one_hot_test_data = one_hot_train_test_data[one_hot_train_test_data.Flag=="test"]

one_hot_train_data.shape,train_data.shape,one_hot_test_data.shape,test_data.shape

((512087, 17), (512087, 12), (36000, 17), (36000, 10))

In [24]:
input_features

['Course_Domain', 'Course_Type', 'Short_Promotion', 'Public_Holiday']

In [27]:
cols_to_exclude = ['Competition_Metric', 'Course_ID', 'Day_No', 'Flag', 'ID','Long_Promotion', 'Sales','User_Traffic']

## Model Preparation

In [29]:
# convert train data to the xgboost train object

xgb_train_data = xgb.DMatrix(one_hot_train_data[one_hot_train_data.columns.difference(cols_to_exclude)],one_hot_train_data[target])
xgb_test_data = xgb.DMatrix(one_hot_test_data[one_hot_test_data.columns.difference(cols_to_exclude)])

In [30]:
# initial training params
train_params = {"objective" : "reg:linear", "eval_metric" : "rmse"}

## train model

In [32]:

xgb_model = xgb.train(params=train_params,dtrain=xgb_train_data)

[18:15:50] WARNING: d:\build\xgboost\xgboost-1.0.1.git\src\objective\regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.


## evaluate model on training data

In [42]:
np.sqrt(mean_squared_error(one_hot_train_data[target],
                           xgb_model.predict(xgb.DMatrix(one_hot_train_data[one_hot_train_data.columns.difference(cols_to_exclude)])))
       )

48.95879660720635

In [43]:
one_hot_train_data.Sales.describe()

count    512087.000000
mean        120.826924
std          54.355258
min           0.000000
25%          84.000000
50%         111.000000
75%         146.000000
max         682.000000
Name: Sales, dtype: float64

In [44]:
one_hot_test_data

,Competition_Metric,Course_ID,Day_No,Flag,ID,Long_Promotion,Public_Holiday,Sales,Short_Promotion,User_Traffic,Course_Domain_Business,Course_Domain_Development,Course_Domain_Finance & Accounting,Course_Domain_Software Marketing,Course_Type_Course,Course_Type_Degree,Course_Type_Program
0,0.007,1,883,test,883,1,0,NaN,1,NaN,0,1,0,0,1,0,0
1,0.007,1,884,test,884,1,0,NaN,1,NaN,0,1,0,0,1,0,0
2,0.007,1,885,test,885,1,0,NaN,1,NaN,0,1,0,0,1,0,0
3,0.007,1,886,test,886,1,0,NaN,1,NaN,0,1,0,0,1,0,0
4,0.007,1,887,test,887,1,0,NaN,0,NaN,0,1,0,0,1,0,0
5,0.007,1,888,test,888,1,0,NaN,0,NaN,0,1,0,0,1,0,0
6,0.007,1,889,test,889,1,0,NaN,0,NaN,0,1,0,0,1,0,0
7,0.007,1,890,test,890,1,0,NaN,0,NaN,0,1,0,0,1,0,0
8,0.007,1,891,test,891,1,0,NaN,0,NaN,0,1,0,0,1,0,0
9,0.007,1,892,test,892,1,0,NaN,0,NaN,0,1,0,0,1,0,0


In [45]:
test_predictions  = xgb_model.predict(xgb_test_data)
test_predictions

array([135.06668, 135.06668, 135.06668, ..., 142.7821 , 142.7821 ,
       142.7821 ], dtype=float32)

In [46]:
one_hot_test_data["Sales"] = test_predictions

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [48]:
one_hot_test_data.isnull().sum()

Competition_Metric                      120
Course_ID                                 0
Day_No                                    0
Flag                                      0
ID                                        0
Long_Promotion                            0
Public_Holiday                            0
Sales                                     0
Short_Promotion                           0
User_Traffic                          36000
Course_Domain_Business                    0
Course_Domain_Development                 0
Course_Domain_Finance & Accounting        0
Course_Domain_Software Marketing          0
Course_Type_Course                        0
Course_Type_Degree                        0
Course_Type_Program                       0
dtype: int64

In [49]:
test_submission = one_hot_test_data[["ID","Sales"]]
test_submission

,ID,Sales
0,883,135.066681
1,884,135.066681
2,885,135.066681
3,886,135.066681
4,887,93.063187
5,888,93.063187
6,889,93.063187
7,890,93.063187
8,891,93.063187
9,892,93.063187


In [51]:
test_submission["Sales"] = round(test_submission["Sales"])
test_submission

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ID,Sales
0,883,135.0
1,884,135.0
2,885,135.0
3,886,135.0
4,887,93.0
5,888,93.0
6,889,93.0
7,890,93.0
8,891,93.0
9,892,93.0


In [52]:
test_submission.to_csv("../data/test_predictions.csv",
                      header=True,
                      index=False)